In [ ]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

gpu_num = 0  # Use "" to use the CPU, Use 0 to select first GPU
os.environ["CUDA_VISIBLE_DEVICES"] = f"{gpu_num}"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# Configure the notebook to use only a single GPU and allocate only as much memory as needed
import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        print(e)
tf.get_logger().setLevel('ERROR')

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
from sionna.channel import AWGN
from dmimo.channel import LoadNs3Channel, estimate_capacity
from dmimo.config import Ns3Config

In [ ]:
ns3_config = Ns3Config(total_slots=11, ue_txpwr_ctrl=False)
ns3_channel = LoadNs3Channel(ns3_config)
add_noise = AWGN()

In [ ]:
def sample_channel(channel_type="dMIMO", slot_idx=5):
    # [batch_size, num_rx, num_rx_ant, num_tx, num_tx_ant, num_ofdm_symbols, fft_size]
    h_freq, snrdb = ns3_channel(channel_type, slot_idx=slot_idx)
    # [batch_size, num_rx_ant, num_tx_ant, num_ofdm_symbols, fft_size]
    h_freq = np.squeeze(h_freq, axis=(1,3))

    # [batch_size, num_tx_ant, num_rx_ant, num_ofdm_symbols, fft_size]
    h_freq = tf.transpose(h_freq, (0, 2, 1, 3, 4))

    return h_freq, snrdb

#### Transmit Squad Channels

In [ ]:
h_txs, snr_txs = sample_channel("TxSquad")
print(h_txs.shape, snr_txs.shape)
print(np.var(h_txs))

In [ ]:
c_txs = estimate_capacity(h_txs, snrdb=20.0)
t_txs = c_txs * 7.68
print("Estimated capacity: {:.2f} bps/Hz, ({:.2f} Mbps for 7.68MHz)".format(c_txs, t_txs))

In [ ]:
im = plt.plot(snr_txs[0,0].transpose())

#### Receiving Squad Channels

In [ ]:
h_rxs, snr_rxs = sample_channel("RxSquad")
print(h_rxs.shape, snr_rxs.shape)
print(np.var(h_rxs))

In [ ]:
c_rxs = estimate_capacity(h_rxs, snrdb=20.0)
t_rxs = c_rxs * 7.68
print("Estimated capacity: {:.2f} bps/Hz, ({:.2f} Mbps for 7.68MHz)".format(c_rxs, t_rxs))

In [ ]:
im = plt.plot(snr_rxs[0,0].transpose())

#### Squad-to-squad dMIMO Channels

In [ ]:
h_dm, snr_dm = sample_channel("dMIMO")
print(h_dm.shape, snr_dm.shape)
print(np.var(h_dm))

In [ ]:
c_dm = estimate_capacity(h_dm, snrdb=10.0)
t_dm = c_dm * 7.68
print("Estimated capacity: {:.2f} bps/Hz, ({:.2f} Mbps for 7.68MHz)".format(c_dm, t_dm))

In [ ]:
im = plt.plot(snr_dm[0,0].transpose())

#### Effective SU-MIMO Channel using Analog Forwarding

In [ ]:
h_fwd, snr_fwd = sample_channel("dMIMO-Forward")
print(h_fwd.shape)
print(np.var(h_fwd[:,:4]))
print(np.var(h_fwd[:,4:]))

In [ ]:
c_fwd = estimate_capacity(h_fwd, snrdb=10.0)
t_fwd = c_fwd * 7.68
print("Estimated capacity: {:.2f} bps/Hz, ({:.2f} Mbps for 7.68MHz)".format(c_fwd, t_fwd))

In [ ]:
im = plt.plot(snr_fwd[0,0].transpose())

#### Frequency-domain Channel Covariance

In [ ]:
#################################
# Estimate frequency covariance
#################################
def estimate_freq_cov(channel_type="dMIMO"):
    fft_size = tf.cast(512, tf.int64)
    freq_cov_mat = tf.zeros([fft_size, fft_size], tf.complex64)

    for iter in tf.range(ns3_config.total_slots):
        # [num_batch, num_tx_ant, num_rx_ant, num_ofdm_symbols, fft_size]
        h_samples, snr_samples = sample_channel(channel_type)

        # [num_batch, num_tx_ant, num_rx_ant, fft_size, num_ofdm_symbols]
        h_samples_ = tf.transpose(h_samples, [0,1,2,4,3])

        # [num_batch, num_tx_ant, num_rx_ant, fft_size, fft_size]
        freq_cov_mat_ = tf.matmul(h_samples_, h_samples_, adjoint_b=True)
        # [fft_size, fft_size]
        freq_cov_mat_ = tf.reduce_mean(freq_cov_mat_, axis=(0,1,2))
        # [fft_size, fft_size]
        freq_cov_mat += freq_cov_mat_

    freq_cov_mat /= tf.complex(tf.cast(10, tf.float32), tf.cast(0.0, tf.float32))

    return freq_cov_mat.numpy()

In [ ]:
freq_cov_dmimo = estimate_freq_cov("dMIMO")

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(10,5))
fig.suptitle("Time and frequency channel covariance matrices")

ax[0].set_title("Freq. cov. Real")
im = ax[0].imshow(freq_cov_dmimo.real)
ax[1].set_title("Freq. cov. Imag")
im = ax[1].imshow(freq_cov_dmimo.imag)

In [ ]:
freq_cov_fwd = estimate_freq_cov("dMIMO-Forward")

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(10,5))
fig.suptitle("Time and frequency channel covariance matrices")

ax[0].set_title("Freq. cov. Real")
im = ax[0].imshow(freq_cov_fwd.real)
ax[1].set_title("Freq. cov. Imag")
im = ax[1].imshow(freq_cov_fwd.imag)

#### Old ns-3 Channel Model without Proper Coefficient Scaling

In [ ]:
h_fwd_old, snr_fwd_old = sample_channel("dMIMO-Simple")
print(h_fwd_old.shape)
print(np.var(h_fwd_old))

In [ ]:
c_fwd_old = estimate_capacity(h_fwd_old, snrdb=10.0)
print("Estimated capacity: {:.2f} bps/Hz, ({:.2f} Mbps for 7.68MHz)".format(c_fwd_old, c_fwd_old * 7.68))

In [ ]:
im = plt.plot(snr_fwd_old[0,0].transpose())

In [ ]:
freq_cov_old = estimate_freq_cov("dMIMO-Simple")

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(10,5))
fig.suptitle("Time and frequency channel covariance matrices")

ax[0].set_title("Freq. cov. Real")
im = ax[0].imshow(freq_cov_old.real) #, vmin=-0.3, vmax=1.8)
ax[1].set_title("Freq. cov. Imag")
im = ax[1].imshow(freq_cov_old.imag) #, vmin=-0.3, vmax=1.8)